In [ ]:
# Se cargan las librerías necesarias para utilizar la red convolucional 
import tensorflow
import keras 
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras import utils
# from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D
# from keras.layers import Conv2D
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split

In [ ]:
# Se generan los path necesarios para cargar las imágenes de entrenamiento y validación 
import os
path_entreno_2 = '/home/cristian/Escritorio/Reconocedor facial/Datos_desarrollo/Entrenamiento_2/'

epochs = 5
altura, ancho = 250, 250
batch_size = 37
filtro_conv_1 = 32
filtro_conv_2 = 64
tamano_fitro_1 = (5,5)
tamano_filtro_2 = (3,3)
tamano_pool = (2,2)
clases = 2
lr = 0.005 

In [ ]:
# Se cargan los datos de entrenamiento y test
images = []
labels = []

for path in range(len(os.listdir(path_entreno_2))):
    directorio = path_entreno_2 + os.listdir(path_entreno_2)[path] + '/'
    for image in range(len(os.listdir(directorio))):
        imagen = cv2.imread(path_entreno_2 + os.listdir(path_entreno_2)[path] + '/' + os.listdir(directorio)[image], cv2.IMREAD_GRAYSCALE)
        images.append(imagen)
        labels.append(path)


In [ ]:
# Se definen los conjuntos de entrenamiento y test
images, labels = np.array(images), np.array(labels) 
labels = utils.to_categorical(labels, clases)

images_train, images_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 1, shuffle = True)

In [ ]:
# Se utiliza la función ImageDataGenerator para poder entrenar mejor las imágenes
images_data_gen_train = ImageDataGenerator(rescale = 1.0,
                                           shear_range = 0.3,
                                           zoom_range = 0.3,
                                           horizontal_flip = True,
                                           vertical_flip = True,
                                           rotation_range = 20)


images_data_gen_test = ImageDataGenerator(rescale = 1.0 / 255)

images_data_gen_train.fit(images_train)
images_data_gen_test.fit(images_test)

In [ ]:
# Se genera el modelo
modelo = Sequential()
modelo.add(Convolution2D(filters = filtro_conv_1,
    batch_size = batch_size,
    kernel_size = tamano_fitro_1,
    padding = 'same',
    input_shape = (altura, ancho, 3),
    activation = 'relu'))
modelo.add(LeakyReLU(alpha = 0.1))
modelo.add(MaxPooling2D(pool_size = tamano_pool))
modelo.add(Dropout(0.2))
modelo.add(Convolution2D(filters= filtro_conv_2,
                  kernel_size = tamano_filtro_2,
                  activation = 'relu'))
modelo.add(MaxPooling2D(pool_size = tamano_pool))
modelo.add(Dropout(0.2))
modelo.add(Flatten())
modelo.add(Dense(128, activation = 'relu'))
modelo.add(Dense(2, activation = 'softmax'))
modelo.compile(loss = keras.losses.categorical_crossentropy,
               optimizer = optimizers.Adagrad(lr = lr, decay = lr/100) ,
               metrics = ['accuracy'])
modelo.summary()

In [ ]:
# Se entrena el modelo 
history = modelo.fit_generator(images_data_gen_train.flow(images_train, y_train, batch_size = batch_size),
                     epochs=epochs, validation_data = images_data_gen_test.flow(images_test, 
                     y_test, batch_size = batch_size), shuffle = True) 


In [ ]:
test_evaluate = modelo.evaluate(images_data_gen_test.flow(images_test, y_test, batch_size = batch_size), verbose = 1)
test_evaluate[0]
test_evaluate[1]